In [1]:
import sys
# sys.path.append('/scratch/pradap/python-work/anhaidgroup/py_entitymatching/')
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/py_entitymatching/')

In [2]:
import py_entitymatching as em
import pandas as pd

In [3]:
import mur_utils as mu

In [4]:
num_pos_diff_year = 0
num_pos_same_year_swapped = 75
num_pos_same_year_not_swapped = 75
num_neg_samples = 300
num_neg_samples_to_be_swapped = 100 #200

In [5]:
A2 = em.read_csv_metadata('ORIG_DBLP_ACM/dblp_mur_diff_year_sim_title_removed.csv', key='id', dtype=str)
B4 = em.read_csv_metadata('ORIG_DBLP_ACM/acm_sim_title_rem_perturb_year.csv',  key='id', dtype=str)
B2 = em.read_csv_metadata('ORIG_DBLP_ACM/acm_mur_diff_year_sim_title_removed.csv',  key='id', dtype=str)


In [6]:
gold = em.read_csv_metadata('ORIG_DBLP_ACM/goldWithLabel.csv', key='_id', fk_ltable='ltable_id', fk_rtable='rtable_id',
                           ltable=A2, rtable=B4, dtype=str)

In [7]:
ab = em.AttrEquivalenceBlocker()
C2 = ab.block_tables(A2, B4, 'year', 'year', ['title', 'authors', 'year'], ['title', 'authors', 'year'])

In [8]:
ob = em.OverlapBlocker()
C3 = ob.block_tables(A2, B4, 'title', 'title', l_output_attrs=['title', 'authors', 'year'], r_output_attrs=['title', 'authors', 'year'])

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


In [9]:
C = em.combine_blocker_outputs_via_union([C2, C3])

In [10]:
count, tuple_ids = mu.get_num_positives_candset(C, gold)
print(count)

1575


In [11]:
g = gold.set_index(['ltable_id', 'rtable_id'], drop=False)
g1 = g.ix[tuple_ids]
em.set_fk_ltable(g1, 'ltable_id')
em.set_fk_rtable(g1, 'rtable_id')
em.set_ltable(g1, A2)
em.set_rtable(g1, B4)
em.set_key(g1, '_id')

True

In [12]:
g1.reset_index(drop=True, inplace=True)
g1 = em.add_output_attributes(g1, l_output_attrs=['year'], r_output_attrs=['year'])
g1.head()
g1.rename(columns={'rtable_year': 'rtable_year_perturbed'}, inplace=True)
em.set_fk_ltable(g1, 'ltable_id')
em.set_fk_rtable(g1, 'rtable_id')
em.set_ltable(g1, A2)
em.set_rtable(g1, B2)
em.set_key(g1, '_id')
g1 = em.add_output_attributes(g1, r_output_attrs=['year'])

In [13]:
g1_same_year = em.filter_rows(g1, 'ltable_year == rtable_year_perturbed')
len(g1_same_year)

1396

In [14]:
g1_diff_year = em.filter_rows(g1, 'ltable_year != rtable_year_perturbed')
len(g1_diff_year)

179

In [15]:
g1_diff_year_sampled = em.sample_table(g1_diff_year, num_pos_diff_year)
g1_same_year_splitted_2 = em.split_train_test(g1_same_year, train_proportion=0.5)
g1_same_year_to_be_swapped_sample = em.sample_table(g1_same_year_splitted_2['train'], num_pos_same_year_swapped)
g1_same_year_not_to_be_swapped_sample = em.sample_table(g1_same_year_splitted_2['test'], num_pos_same_year_not_swapped)
len(g1_same_year_to_be_swapped_sample), len(g1_same_year_not_to_be_swapped_sample), len(g1_diff_year_sampled)

(75, 75, 0)

In [16]:
len(g1_same_year_to_be_swapped_sample.ltable_id.values)

75

In [17]:
A3 = A2.set_index('id', drop=False)
titles = A3.ix[g1_same_year_to_be_swapped_sample.ltable_id.values, 'title']
authors = A3.ix[g1_same_year_to_be_swapped_sample.ltable_id.values, 'authors']
A4 = A3.copy()
A4.ix[g1_same_year_to_be_swapped_sample.ltable_id.values, 'authors'] = titles
A4.ix[g1_same_year_to_be_swapped_sample.ltable_id.values, 'title'] = authors
A4.ix[g1_same_year_to_be_swapped_sample.ltable_id.values].head()

,id,title,authors,venue,year
id,,,,,
conf/sigmod/GionisGK01,conf/sigmod/GionisGK01,"Dimitrios Gunopulos, Aristides Gionis, Nick Koudas",Efficient and Tunable Similar Set Retrieval,SIGMOD Conference,2001
conf/sigmod/Moore95,conf/sigmod/Moore95,Kenneth Moore,The Lotus Notes Storage System,SIGMOD Conference,1995
journals/sigmod/Kim94,journals/sigmod/Kim94,Won Kim,Observations on the ODMG-93 Proposal,SIGMOD Record,1994
conf/vldb/ZurekS99,conf/vldb/ZurekS99,"Thomas Zurek, Markus Sinnwell",Datawarehousing Has More Colours Than Just Black & White,VLDB,1999
journals/sigmod/Gardels97,journals/sigmod/Gardels97,Kenn Gardels,Open GIS and On-Line Environmental Libraries,SIGMOD Record,1997


In [18]:
em.set_key(A4, 'id')
A4.reset_index(inplace=True, drop=True)

In [19]:
em.to_csv_metadata(A4, 'ORIG_DBLP_ACM/dblp_sim_title_rem_title_auth_swapped.csv')

True

In [20]:
A5 = A4.set_index('id', drop=False)
# l_output_attrs=['title', 'authors', 'year']

In [21]:
indices_not_swapped = set(A5.index.values).difference(g1_same_year_to_be_swapped_sample.ltable_id.values)

In [22]:
len(indices_not_swapped)

1725

In [23]:
count, a_indices, b_indices = mu.get_num_positives_tables(A2, B4, gold)

In [24]:
count, len(a_indices), len(A2)

(1575, 1575, 1800)

In [25]:
A6 = A5.ix[set(A5.index.values).difference(a_indices)]

In [26]:
A6.dropna(inplace=True)

In [27]:
B3 = B2.set_index('id', drop=False)

In [28]:
B4 = B3.ix[set(B3.index.values).difference(b_indices)]

In [29]:
B4.dropna(inplace=True)

In [30]:
len(B4), len(A6)

(234, 225)

In [31]:
em.set_key(B4, 'id')
em.set_key(A6, 'id')

True

In [32]:
C5 = ab.block_tables(A6, B4, 'year', 'year', l_output_attrs=['title', 'authors', 'year'], 
                     r_output_attrs=['title', 'authors', 'year'])

In [33]:
len(C5)

5165

In [34]:
C6 = em.sample_table(C5, num_neg_samples)
d1 = C6

In [35]:
neg_samples_in_A_to_be_swapped = list(pd.unique(C6['ltable_id'].values))

In [36]:
import numpy as np
neg_samples_in_A_to_be_swapped_sample = np.random.choice(neg_samples_in_A_to_be_swapped, 
                                                         num_neg_samples_to_be_swapped)


In [37]:
len(neg_samples_in_A_to_be_swapped_sample)

100

In [38]:
titles = A6.ix[neg_samples_in_A_to_be_swapped_sample, 'title']
authors = A6.ix[neg_samples_in_A_to_be_swapped_sample, 'authors']
A7 = A6.copy()
A7.ix[neg_samples_in_A_to_be_swapped_sample, 'authors'] = titles
A7.ix[neg_samples_in_A_to_be_swapped_sample, 'title'] = authors
A7.ix[neg_samples_in_A_to_be_swapped_sample].head()

,id,title,authors,venue,year
id,,,,,
conf/sigmod/Aulakh98,conf/sigmod/Aulakh98,Kamar Aulakh,About Quark Digital Media System,SIGMOD Conference,1998
conf/vldb/BaulierBGJGKKMMNNRSS98,conf/vldb/BaulierBGJGKKMMNNRSS98,"P. P. S. Narayan, S. Joshi, M. Nemeth, Abraham Silberschatz, Henry F. Korth, A. Khivesera, Jerry...",DataBlitz: A High Performance Main-Memory Storage Manager,VLDB,1998
conf/sigmod/Aggarwal03,conf/sigmod/Aggarwal03,Charu C. Aggarwal,A Framework for Change Diagnosis of Data Streams,SIGMOD Conference,2003
conf/sigmod/GangulyGR03,conf/sigmod/GangulyGR03,"Sumit Ganguly, Minos N. Garofalakis, Rajeev Rastogi",Processing Set Expressions over Continuous Update Streams,SIGMOD Conference,2003
journals/tods/WangBBJ97,journals/tods/WangBBJ97,"Alexander Brodsky, Sushil Jajodia, Claudio Bettini, Xiaoyang Sean Wang",Logical Design for Temporal Databases with Multiple Granularities,ACM Trans. Database Syst.,1997


In [39]:
em.set_key(A7, 'id')
A7.reset_index(inplace=True, drop=True)

In [40]:
em.to_csv_metadata(A4, 'ORIG_DBLP_ACM/dblp_sim_title_rem_title_auth_swapped_for_neg_samples_too.csv')

True

In [41]:
len(d1)

300

In [42]:
d2 = d1[['_id', 'ltable_id', 'rtable_id']]

In [43]:
len(g1_diff_year_sampled), len(g1_same_year_not_to_be_swapped_sample), len(g1_same_year_to_be_swapped_sample), len(d2)

(0, 75, 75, 300)

In [44]:
g1_diff_year_sampled_prj = g1_diff_year_sampled[['_id', 'ltable_id', 'rtable_id', 'label']]
g1_same_year_not_to_be_swapped_sample = g1_same_year_not_to_be_swapped_sample[['_id', 'ltable_id', 'rtable_id', 'label']]
g1_same_year_to_be_swapped_sample = g1_same_year_to_be_swapped_sample[['_id', 'ltable_id', 'rtable_id', 'label']]

In [45]:
em.filter_rows(d1, 'ltable_year==rtable_year')

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_year,rtable_title,rtable_authors,rtable_year
14,14,conf/vldb/MillerHH00,336573,Schema Mapping as Query Discovery,"Rene J. Miller, Mauricio A. Hernndez, Laura M. Haas",2000,Image mining in IRIS: integrated retinal information system,"Wynne Hsu, Mong Li Lee, Kheng Guan Goh",2000
59,59,conf/sigmod/Hellerstein00,672006,Index Research: Forest or Trees? (Panel Abstract),Joseph M. Hellerstein,2000,Temporal Queries in OLAP,"Alberto O. Mendelzon, Alejandro A. Vaisman",2000
127,127,journals/sigmod/LabrinidisR00,362137,Generating Dynamic Content at Database-Backed Web Servers: cgi-bin vs. mod_perl,"Alexandros Labrinidis, Nick Roussopoulos",2000,The implementation and performance of compressed databases,"Till Westmann, Donald Kossmann, Sven Helmer, Guido Moerkotte",2000
131,131,journals/sigmod/LabrinidisR00,335395,Generating Dynamic Content at Database-Backed Web Servers: cgi-bin vs. mod_perl,"Alexandros Labrinidis, Nick Roussopoulos",2000,On wrapping query languages and efficient XML integration,"Vassilis Christophides, Sophie Cluet, Jérǒme Simèon",2000
133,133,journals/sigmod/LabrinidisR00,672019,Generating Dynamic Content at Database-Backed Web Servers: cgi-bin vs. mod_perl,"Alexandros Labrinidis, Nick Roussopoulos",2000,Novel Approaches in Query Processing for Moving Object Trajectories,"Dieter Pfoser, Christian S. Jensen, Yannis Theodoridis",2000
136,136,journals/sigmod/LabrinidisR00,765224,Generating Dynamic Content at Database-Backed Web Servers: cgi-bin vs. mod_perl,"Alexandros Labrinidis, Nick Roussopoulos",2000,UnQL: a query language and algebra for semistructured data based on structural recursion,"Peter Buneman, Mary Fernandez, Dan Suciu",2000
166,166,conf/sigmod/RoySSB00,765224,Efficient and Extensible Algorithms for Multi Query Optimization,"S. Sudarshan, Siddhesh Bhobe, S. Seshadri, Prasan Roy",2000,UnQL: a query language and algebra for semistructured data based on structural recursion,"Peter Buneman, Mary Fernandez, Dan Suciu",2000
179,179,conf/sigmod/RoySSB00,672006,Efficient and Extensible Algorithms for Multi Query Optimization,"S. Sudarshan, Siddhesh Bhobe, S. Seshadri, Prasan Roy",2000,Temporal Queries in OLAP,"Alberto O. Mendelzon, Alejandro A. Vaisman",2000
197,197,conf/vldb/CasatiS00,335437,Process Automation as the Foundation for E-Business,"Fabio Casati, Ming-Chien Shan",2000,Efficient algorithms for mining outliers from large data sets,"Sridhar Ramaswamy, Rajeev Rastogi, Kyuseok Shim",2000
200,200,conf/vldb/CasatiS00,758379,Process Automation as the Foundation for E-Business,"Fabio Casati, Ming-Chien Shan",2000,Building and Customizing Data-Intensive Web Sites Using Weave,"Khaled Yagoub, Daniela Florescu, Valérie Issarny, Patrick Valduriez",2000


In [46]:
d2['label'] = 0

/Users/pradap/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [47]:
labeled_data = pd.concat([g1_diff_year_sampled_prj, g1_same_year_not_to_be_swapped_sample, 
                          g1_same_year_to_be_swapped_sample, d2], ignore_index=True)


In [48]:
sum(labeled_data.label.astype(int))

150

In [49]:
labeled_data = labeled_data.sample(frac=1)
labeled_data['_id'] = range(0, len(labeled_data))
labeled_data.reset_index(inplace=True)
labeled_data = labeled_data[['_id', 'ltable_id', 'rtable_id', 'label']]

In [50]:
em.set_key(labeled_data, '_id')
em.set_ltable(labeled_data, A4)
em.set_rtable(labeled_data, B4)
em.set_fk_ltable(labeled_data, 'ltable_id')
em.set_fk_rtable(labeled_data, 'rtable_id')

True

In [51]:
em.to_csv_metadata(labeled_data, 'ORIG_DBLP_ACM/labeled_data_dblp_acm_after_monkey_tricks.csv')

True

In [52]:
len(labeled_data)

450